In [1]:
import pandas as p

url = "SMSSpamCollection.txt"
dataset = p.read_table(url, encoding='latin-1', header = None)

In [2]:
print(dataset.shape)

(5572, 2)


In [3]:
dataset.head(10)

,0,1
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
print(dataset.groupby(0).size())

0
ham     4825
spam     747
dtype: int64


Displaying the proportion of spam and ham in the dataset taken from kaggle

In [5]:
import matplotlib.pyplot as plt

# Data to plot
labels = 'Spam', 'Ham'
sizes = [len(dataset[dataset[0] == 'spam']), len(dataset[dataset[0] == 'ham'])]
colors = ['gold', 'green']
explode = (0.2, 0)

# Plot
plt.pie(sizes, explode=explode, labels=labels, colors=colors,autopct='%1.1f%%', shadow=True, startangle=90)

plt.axis('equal')
plt.show()

<Figure size 640x480 with 1 Axes>

Make the sms in lower case
Removal of the punctuations
Removal of stop-words (a, an, of, until...)

In [6]:
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def smsTransformation(message):
    message = message.translate(str.maketrans('', '', string.punctuation))
    word = [words.lower() for words in word_tokenize(message) if words.lower() not in stopwords.words("english")
            and words.lower().isalpha()]
    message = ' '.join(word)
    return message

In [7]:
for i in range(5571):
    dataset.loc[i, 1] = smsTransformation(dataset.loc[i, 1])

In [8]:
dataset.head(10)

,0,1
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry wkly comp win fa cup final tkts may...
3,ham,u dun say early hor u c already say
4,ham,nah dont think goes usf lives around though
5,spam,freemsg hey darling weeks word back id like fu...
6,ham,even brother like speak treat like aids patent
7,ham,per request melle melle oru minnaminunginte nu...
8,spam,winner valued network customer selected receiv...
9,spam,mobile months u r entitled update latest colou...


the words are referenced by numbers, same for the sms row
It is having a set of (lineNO, wordNO)
--the whole logic is not understood, the technique is taken from kaggle--

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = TfidfVectorizer(encoding = "latin-1")
vectorizer = CountVectorizer()
features = vectorizer.fit_transform(dataset[1])

In [10]:
#print(features.shape)
#print(vectorizer.get_feature_names())
features

<5572x8127 sparse matrix of type '<class 'numpy.int64'>'
	with 44485 stored elements in Compressed Sparse Row format>

In [11]:
print(features.toarray())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
from sklearn.preprocessing import LabelEncoder

category = LabelEncoder().fit_transform(dataset[0])

In [13]:
from sklearn.model_selection import train_test_split

xTrain, xValidation, yTrain, yValidation = train_test_split(features, category, test_size = 0.2, random_state = 42)

In [14]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

gaussianNb = MultinomialNB()
gaussianNb.fit(xTrain, yTrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
prediction = gaussianNb.predict(xValidation)
accuracy_score(yValidation, prediction) 

0.9632286995515695

In [16]:
from sklearn.metrics import classification_report

print(classification_report(yValidation, prediction))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       966
           1       0.82      0.93      0.87       149

    accuracy                           0.96      1115
   macro avg       0.90      0.95      0.92      1115
weighted avg       0.97      0.96      0.96      1115



In [17]:
def smsFilter(message):
    message = smsTransformation(message)
    form = vectorizer.transform([message])
    prediction = gaussianNb.predict(form)
    if (prediction):
        return 'spam'
    else:
        return 'ham'
   
    
smsFilter('Uber Eats: Hanging out with friends? Order in what you crave. Use code UBDR1 to get 50% off 4 orders upto Rs.75, TCA. Order now: t.uber.com/leq')

'spam'

In [18]:
from flask import Flask, redirect, url_for, request
app = Flask(__name__)

@app.route('/success/<name>')
def success(name):
   return '''It's a %s''' % name

@app.route('/login',methods = ['POST', 'GET'])
def login():
    if request.method == 'POST':
        message = request.form['message']
        answer = smsFilter(message)
    return redirect(url_for('success',name = answer))

if __name__ == '__main__':
   app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Dec/2019 11:37:30] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [01/Dec/2019 11:37:30] "GET /success/ham HTTP/1.1" 200 -
